In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
ls gdrive/MyDrive/

 books_recommender_chatbot/  'Colab Notebooks'/   cv/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
books = pd.read_csv('/content/gdrive/MyDrive/books_recommender_chatbot/goodreads1-100.csv', encoding='Latin-1')
display(books.head())

,title,original_title,series,language,authors,avg_rating,num_ratings,num_reviews,genres,description,url
0,A Prayer for Owen Meany,A Prayer for Owen Meany,NaN,English,John Irving,4.23,294351,14399,"Fiction,Classics,Contemporary,Literature,Novel...","Eleven-year-old Owen Meany, playing in a Littl...",https://www.goodreads.com/book/show/4473.A_Pra...
1,The World According to Garp,The World According to Garp,NaN,English,John Irving,4.09,210460,5716,"Fiction,Classics,Contemporary,Literature,Novel...","This is the life and times of T. S. Garp, the ...",https://www.goodreads.com/book/show/7069.The_W...
2,Leaves of Grass,Leaves of Grass,Iowa Whitman Series,English,Walt Whitman,4.11,91893,2640,"Poetry,Classics,Fiction,Literature,Literature,...",A collection of quintessentially American poem...,https://www.goodreads.com/book/show/27494.Leav...
3,The Firm,The Firm,NaN,English,"Robin Waterfield (Adapter),John Grisham",4.03,588541,3604,"Fiction,Thriller,Mystery,Suspense,Mystery,Crim...","Adaptation for younger readers. Mitch McDeere,...",https://www.goodreads.com/book/show/5358.The_Firm
4,The Last Battle,The Last Battle,The Chronicles of Narnia (Publication Order),English,C.S. Lewis,4.01,240167,7167,"Fantasy,Fiction,Classics,Young Adult,Childrens...",This edition of Lewis's classic fantasy fictio...,https://www.goodreads.com/book/show/84369.The_...


Just as a reminder, here are the columns with the top two rows for example:

* `title` - The title of the book in the language of this particular edition.
* `original_title` - The title in the original language (this is usually the same as `title`).
* `series` - The name of the series the book belongs to, if any.
* `authors` - A comma separated list of the contributing authors.  Roles are sometimes listed in (brackets) after their name.
* `avg_rating` - The average rating out of 5 that the book has across all ratings.
* `num_ratings` - How many times was the book rated (by which `avg_rating` was calculated from).
* `num_reviews` - How many times was the book reviewed?
* `genres` - A comma separated list of the top genres listed on this book's page.
* `description` - The description of the book on the book's page, similar to the blurb.
* `url` - The url of page where you can find the books.


In [ ]:
print(f'Dataframe Dimensions: \nData Points:{books.shape[0]},  Attributes:{books.shape[1]}')

Dataframe Dimensions:
Data Points:10000,  Attributes:11


In [ ]:
display(books.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           10000 non-null  object 
 1   original_title  8912 non-null   object 
 2   series          5167 non-null   object 
 3   language        9683 non-null   object 
 4   authors         10000 non-null  object 
 5   avg_rating      10000 non-null  float64
 6   num_ratings     10000 non-null  int64  
 7   num_reviews     10000 non-null  int64  
 8   genres          9879 non-null   object 
 9   description     9933 non-null   object 
 10  url             10000 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 859.5+ KB


None

In [ ]:
display(books.isna().sum().sort_values(ascending=False))

series            4833
original_title    1088
language           317
genres             121
description         67
url                  0
num_reviews          0
num_ratings          0
avg_rating           0
authors              0
title                0
dtype: int64

Note: 
- The original_title is not always available. 
- Books are not always in series. 
- Language isn't always specified. 
- Titles and urls should always be available and valid.
- Genres and Description warrant further investigation since ideally these should be available.

In [ ]:
books[books.genres.isna()].head(10).url.values

array(['https://www.goodreads.com/book/show/40941784',
       'https://www.goodreads.com/book/show/57213211-desert-outcast',
       'https://www.goodreads.com/book/show/20363899-happy-halloween',
       'https://www.goodreads.com/book/show/57304929-fairy-fairy-quite-contrary',
       'https://www.goodreads.com/book/show/54747752-not-okay',
       'https://www.goodreads.com/book/show/50926309-100-bible-facts-about-jesus',
       'https://www.goodreads.com/book/show/45719695-jubal-s-field-trip-to-heaven',
       'https://www.goodreads.com/book/show/45031654-100-fun-bible-facts',
       'https://www.goodreads.com/book/show/55961601-mad-love',
       'https://www.goodreads.com/book/show/52387868-diya-aur-baati-hum'],
      dtype=object)

In [ ]:
books[books.description.isna()].head(10).url.values

array(['https://www.goodreads.com/book/show/36590307-unblinded',
       'https://www.goodreads.com/book/show/42038.Shakespeare_s_Sonnets',
       'https://www.goodreads.com/book/show/1923820.The_Holy_Bible',
       'https://www.goodreads.com/book/show/323355.The_Book_of_Mormon',
       'https://www.goodreads.com/book/show/6295.Howl_and_Other_Poems',
       'https://www.goodreads.com/book/show/2284803.Holy_Bible',
       'https://www.goodreads.com/book/show/56040881-packfire',
       'https://www.goodreads.com/book/show/316943.Selected_Poems',
       'https://www.goodreads.com/book/show/52309.The_Autobiography_of_Benjamin_Franklin',
       'https://www.goodreads.com/book/show/12274389-the-dark-tower-series-collection'],
      dtype=object)

# Genres 
Exploring genres: Looking into which genres exist and how many books are in each category.

In [ ]:
# splitting string in genre
split_genres = books.loc[books.genres.notnull(), 'genres'].str.split(',', expand=True).stack().reset_index(level=-1, drop=True)
split_genres.unique()

array(['Fiction', 'Classics', 'Contemporary', 'Literature', 'Novels',
       'Literary Fiction', 'Historical', 'Historical Fiction',
       'Young Adult', 'Coming Of Age', 'Adult Fiction', 'American',
       'Humor', 'Drama', 'Poetry', '19th Century', 'Philosophy',
       'Classic Literature', 'Academic', 'School',
       'The United States Of America', 'Thriller', 'Mystery', 'Suspense',
       'Crime', 'Mystery Thriller', 'Legal Thriller', 'Law', 'Fantasy',
       'Childrens', 'Christian', 'Adventure', 'Middle Grade',
       'Christian Fiction', 'Science Fiction Fantasy', 'Romance',
       'New Adult', 'Contemporary Romance', 'Womens Fiction', 'Chick Lit',
       'High School', 'Realistic Fiction', 'Nonfiction', 'Autobiography',
       'Memoir', 'Inspirational', 'Spirituality', 'Writing', 'Essays',
       'Self Help', 'Biography', 'Biography Memoir', 'College', 'Warfare',
       'Fighters', 'Love', 'Book Club', 'Epic', 'Adult', 'Paranormal',
       'Vampires', 'Urban Fantasy', 'Parano

Note : Some of these so-called 'genres' should be considered more as 'user tags' but dealing with user data, such problems are inevitable and expected.

In [ ]:
genre_counts = split_genres.value_counts().sort_values(ascending=False)
genre_counts.head(50)

Fantasy                    10760
Fiction                     8069
Young Adult                 5817
Romance                     5657
Literature                  3678
Historical                  3541
Paranormal                  2894
Mystery                     2862
Contemporary                2620
Novels                      2568
Childrens                   2566
Classics                    2534
Science Fiction             2465
Audiobook                   2341
Adult                       2139
Adventure                   2107
Thriller                    1986
Historical Fiction          1939
Cultural                    1839
Magic                       1279
European Literature         1246
Nonfiction                  1134
Humor                       1122
Adult Fiction               1101
Urban Fantasy               1051
Science Fiction Fantasy      997
Supernatural                 992
Middle Grade                 915
Drama                        879
Teen                         856
Horror    

In [ ]:
fig = px.bar(x=genre_counts[:20], y=genre_counts.index[:20], template='plotly_dark', orientation='h', 
             color=genre_counts.index[:20], color_discrete_sequence=px.colors.sequential.RdBu, title='Top 20 Genres', )

fig.update_layout(xaxis_title='Size', yaxis_title='Genre',)
             
fig.show(renderer='colab')

In [ ]:
fig = px.bar(x=genre_counts[-20:], y=genre_counts.index[-20:], template='plotly_dark', orientation='h', 
             color=genre_counts.index[-20:], color_discrete_sequence=px.colors.sequential.RdBu, title='Bottom 20 Genres', )

fig.update_layout(xaxis_title='Size', yaxis_title='Genre',)
             
fig.show(renderer='colab')

# Authors:


In [ ]:
# splitting string in genre
authors = books.loc[books.authors.notnull(), 'authors'].str.split(',', expand=True).stack().reset_index(level=-1, drop=True)
authors.unique()

array(['John Irving', 'Walt Whitman', 'Robin Waterfield (Adapter)', ...,
       'R.A. McDonald', 'Daniel Pyle', 'Maggie Thom'], dtype=object)

In [ ]:
author_counts = authors.value_counts().sort_values(ascending=False)
author_counts.head(10)

Stephen King              67
Terry Pratchett           43
James Patterson           38
Meg Cabot                 38
Rick Riordan              35
Erin Hunter               32
Jennifer L. Armentrout    30
Cassandra Clare           28
Laurell K. Hamilton       28
William Shakespeare       27
dtype: int64

In [ ]:
fig = px.bar(x=author_counts[:20], y=author_counts.index[:20], template='plotly_dark', orientation='h', 
             color=author_counts.index[:20], color_discrete_sequence=px.colors.qualitative.Antique, title='Top 20 Popular Authors', )

fig.update_layout(xaxis_title='Size', yaxis_title='Author',)
             
fig.show(renderer='colab')

# Data Preprocessing

In [ ]:
# dropping duplicated book entries
books.drop_duplicates(subset='title', inplace=True)
books.shape

(9753, 11)

# Weighted ratings: 
Note: 
- Ratings may be imbalanced and therefore becomes necessary to weigh them
- I will be using Imdb's formular for the weighing algorithm for this part:
        Weighted Rating (WR) = (𝑣𝑣+𝑚⋅𝑅)+(𝑚𝑣+𝑚⋅𝐶)

        where:
        𝑅  is the average rating for the item(mean).
        𝑣  is the number of votes for the item.
        𝑚  is the minimum number of votes to be listed.
        𝐶  is the mean vote across the whole report.

In [ ]:
m = books.num_ratings.quantile(0.1)
C = books.avg_rating.mean()

def weighted_rating(book, m, C):
    R = book.avg_rating
    v = book.num_ratings
    
    return (v / (v + m) * R) + (m / (m + v) * C)

In [ ]:
# Calculate the weighted rating for books that are within our threshold.
books.loc[books.num_ratings > m, 'weighted_rating']  = books.loc[books.num_ratings > m].apply(lambda x: weighted_rating(x, m, C), axis=1)

In [ ]:
# Fill the NaN values (i.e., books lower than our threshold) with a zero score.
books['weighted_rating'].fillna(0, inplace=True)

In [ ]:
books.sort_values('weighted_rating', ascending=False).head(10)

,title,original_title,series,language,authors,avg_rating,num_ratings,num_reviews,genres,description,url,weighted_rating
8200,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",NaN,NaN,English,"J.K. Rowling,Mary GrandPrÃ© (Illustrator)",4.79,63408,186,"Fantasy,Young Adult,Fiction,Fantasy,Magic,Adve...",Box Set containing Harry Potter and the Sorcer...,https://www.goodreads.com/book/show/8.Harry_Po...,4.762718
1440,The Complete Calvin and Hobbes,The Complete Calvin and Hobbes,Calvin and Hobbes,English,Bill Watterson,4.79,37095,1059,"Sequential Art,Comics,Humor,Sequential Art,Gra...",[ Box Set | Book One | Book Two | Book Three...,https://www.goodreads.com/book/show/24812.The_...,4.744559
1078,Words of Radiance,Words of Radiance,The Stormlight Archive,English,Brandon Sanderson,4.74,236941,14189,"Fantasy,Fiction,Fantasy,Epic Fantasy,Fantasy,H...","Words of Radiance, Book Two of the Stormlight ...",https://www.goodreads.com/book/show/17332218-w...,4.733005
1297,Harry Potter Series Box Set,NaN,Harry Potter,English,J.K. Rowling,4.72,263321,7608,"Fantasy,Young Adult,Fiction,Childrens,Fantasy,...","Over 4000 pages of Harry Potter and his world,...",https://www.goodreads.com/book/show/862041.Har...,4.713883
7938,Know My Name,Know My Name,NaN,English,Chanel Miller,4.72,93291,13735,"Nonfiction,Autobiography,Memoir,Feminism,Audio...",She was known to the world as Emily Doe when s...,https://www.goodreads.com/book/show/50196744-k...,4.703017
2966,Harry Potter Collection,"Harry Potter Collection (Harry Potter, #1-6)",Harry Potter,English,J.K. Rowling,4.73,31412,924,"Fantasy,Fiction,Young Adult,Fantasy,Magic,Chil...","Six years of magic, adventure, and mystery mak...",https://www.goodreads.com/book/show/10.Harry_P...,4.681253
6273,It's a Magical World,It's a Magical World,Calvin and Hobbes,English,Bill Watterson,4.71,27316,375,"Sequential Art,Comics,Humor,Sequential Art,Gra...",When cartoonist Bill Watterson announced that ...,https://www.goodreads.com/book/show/24814.It_s...,4.656181
57,The Harry Potter Collection 1-4,Harry Potter Boxed Set Books 1-4,Harry Potter,English,"J.K. Rowling,Mary GrandPrÃ© (Illustrator)",4.68,58826,309,"Fantasy,Fiction,Young Adult,Fantasy,Magic,Adve...","The exciting tales of Harry Potter, the young ...",https://www.goodreads.com/book/show/99298.The_...,4.655056
2638,Mark of the Lion Trilogy,Mark of the Lion Trilogy,Mark of the Lion,English,Francine Rivers,4.77,11749,811,"Christian Fiction,Historical,Historical Fictio...",This best-selling trilogy chronicles a tale of...,https://www.goodreads.com/book/show/95602.Mark...,4.646817
2329,ESV Study Bible,NaN,NaN,English,"Anonymous,Lane T. Dennis (Editor),Wayne Grudem...",4.75,13655,323,"Christian,Religion,Theology,Reference,Religion...",The ESV Study Bible was designed to help you u...,https://www.goodreads.com/book/show/5031805-es...,4.644437


In [ ]:
books.sort_values('weighted_rating', ascending=False).tail(10)

,title,original_title,series,language,authors,avg_rating,num_ratings,num_reviews,genres,description,url,weighted_rating
6112,The Tragedy of Man,Az ember tragÃ©diÃ¡ja,NaN,NaN,Imre MadÃ¡ch,4.22,2091,52,"European Literature,Hungarian Literature,Class...",A literary work by the Hungarian author Imre M...,https://www.goodreads.com/book/show/651934.The...,0.0
6111,The Case of the Cosmological Killer: The Rendl...,NaN,Displaced Detective,English,Stephanie Osborn,4.18,49,4,Mystery,"In 1980, RAF Bentwaters and Woodbridge were pl...",https://www.goodreads.com/book/show/15894829-t...,0.0
9172,The Life of Our Lord: Written for His Children...,The Life of Our Lord,NaN,English,Charles Dickens,4.00,2133,426,"Classics,Holiday,Christmas,Religion,Nonfiction...","In this charming, simple retelling of the life...",https://www.goodreads.com/book/show/5342.The_L...,0.0
6108,Secret Deceptions,Secret Deceptions,Amatore's Restaurant,English,James Sillwood,4.47,49,10,Romance,Amatore's Restaurant: The international cast o...,https://www.goodreads.com/book/show/25001951-s...,0.0
9174,"That, Which Clouds the Soul",NaN,NaN,English,Nicolas Ryan Moore,4.43,28,1,NaN,The Pious Commonwealth rules within the walls ...,https://www.goodreads.com/book/show/34813838-t...,0.0
6105,Black Lamb and Grey Falcon,Black Lamb and Grey Falcon,NaN,English,"Rebecca West,Christopher Hitchens (Introduction)",4.22,2091,317,"History,Travel,Nonfiction,Classics,Politics,Au...","Written on the brink of World War II, Rebecca ...",https://www.goodreads.com/book/show/12880.Blac...,0.0
6079,Saga of Darren Shan Box Set,NaN,Cirque du Freak,English,Darren Shan,4.55,2338,88,"Fantasy,Fiction,Horror,Childrens,Science Ficti...",NaN,https://www.goodreads.com/book/show/588704.Sag...,0.0
6076,The Complete Malazan Book of the Fallen,NaN,Malazan Book of the Fallen,English,Steven Erikson,4.72,1827,115,"Fantasy,Fiction,Fantasy,Epic Fantasy,Epic,Fant...",Included in The Malazan Empire collection are ...,https://www.goodreads.com/book/show/20345548-t...,0.0
6065,The Case of the Cosmological Killer: Endings a...,NaN,Displaced Detective,English,Stephanie Osborn,4.26,50,5,"Mystery,Fantasy",After the revelations in The Rendlesham Incide...,https://www.goodreads.com/book/show/16217729-t...,0.0
5808,Falsely Accused of the Unthinkable,NaN,NaN,English,Glenis Kellet,4.35,328,9,"Drama,Novels,Epic,Fiction,Contemporary",An escaped prisoner encounters horrendous stru...,https://www.goodreads.com/book/show/54766656-f...,0.0


In [94]:
fig = px.histogram(books, x='avg_rating', title='Distribution of Average User Ratings', template='plotly_dark', )

fig.show(renderer = 'colab')

In [93]:
fig = px.histogram(books, x='weighted_rating', title='Distribution of Weighted User Ratings', template='plotly_dark', 
                   color_discrete_sequence=['indianred'] )

fig.show(renderer = 'colab')

In [ ]:
books.to_csv('/content/gdrive/MyDrive/books_recommender_chatbot/goodread_books.csv')